In [15]:
import os
import torch
from pprint import pprint


import src.invariant.invariant
import src.instrumentor.variable
from src.instrumentor.variable import VariableInstance
from src.invariant.invariant import SingleInvariantConstant, MultiInvariantConsistency

# reload the invariant module
import importlib
importlib.reload(src.invariant.invariant)
importlib.reload(src.instrumentor.variable)
from src.instrumentor.variable import VariableInstance
from src.invariant.invariant import SingleInvariantConstant, MultiInvariantConsistency

In [2]:
# trace loading process
path_to_traces = "/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing"

trace_paths = [os.path.join(path_to_traces, f) for f in os.listdir(path_to_traces) if f.endswith("_trace.log")]
print(trace_paths)

with open(trace_paths[0], "r") as f:
    lines = f.readlines()

import tqdm, json, pandas as pd
traces = []
for trace in tqdm.tqdm(lines):
    if not trace.startswith("{"):
        # heuristics to skip non-json lines
        continue
    try:
        traces.append(json.loads(trace))
    except:
        print(trace)
        raise

traces_df = pd.DataFrame(traces)
traces_state_change = traces_df[(traces_df["type"] == "state_change")]
trace_state_dump = traces_df[(traces_df["type"] == "state_dump")]
# split the traces by name
traces_by_name = {}
for name, group in traces_state_change.groupby("name"):
    traces_by_name[name] = group.reset_index(drop=True)
    print(name, len(group))

['/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736348_140359961978688_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736346_140282190907200_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736344_140505828259648_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736343_140216820397888_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736345_140158616139584_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736350_139924523960128_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736349_140620820686656_trace.log', '/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing/736347_140014710134592_trace.log']


 16%|█▌        | 328553/2118043 [00:03<00:22, 79285.65it/s] Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8713f6f3d0>>
Traceback (most recent call last):
  File "/home/yuxuan/miniconda3/envs/deepspeed-yuxuan/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
100%|██████████| 2118043/2118043 [00:23<00:00, 89659.03it/s] 


module.3.input_layernorm.bias 23
module.3.input_layernorm.weight 1
module.3.mlp.dense_4h_to_h.bias 23
module.3.mlp.dense_4h_to_h.weight 23
module.3.mlp.dense_h_to_4h.bias 23
module.3.mlp.dense_h_to_4h.weight 23
module.3.post_attention_layernorm.bias 23
module.3.post_attention_layernorm.weight 1
module.3.self_attention.dense.bias 23
module.3.self_attention.dense.weight 23
module.3.self_attention.query_key_value.bias 23
module.3.self_attention.query_key_value.weight 23
module.4.input_layernorm.bias 23
module.4.input_layernorm.weight 1
module.4.mlp.dense_4h_to_h.bias 23
module.4.mlp.dense_4h_to_h.weight 23
module.4.mlp.dense_h_to_4h.bias 23
module.4.mlp.dense_h_to_4h.weight 23
module.4.post_attention_layernorm.bias 23
module.4.post_attention_layernorm.weight 1
module.4.self_attention.dense.bias 23
module.4.self_attention.dense.weight 23
module.4.self_attention.query_key_value.bias 23
module.4.self_attention.query_key_value.weight 23
module.5.input_layernorm.bias 23
module.5.input_layernor

In [7]:
def construct_states(initial_state, changes):
    # change "param" to "value" for consistency
    if 'param' in initial_state:
        initial_state['value'] = initial_state['param']
        del initial_state['param']
    
    states = [initial_state]
    for i, trace in changes.iterrows():
        state = states[-1].copy()
        if 'properties' in trace.change:
            state['properties'] = trace.change['properties']['new']
        if 'value' in trace.change:
            state['value'] = trace.change['value']['new']
        states.append(state)
    return states

layer = 'module.3.mlp.dense_h_to_4h.bias'
state_changes = traces_by_name[layer]
initial_state = None
for param in trace_state_dump.iloc[0].state:
    if param['name'] == layer:
        initial_state = param
        break
assert initial_state is not None
states = construct_states(initial_state, state_changes)
vi = VariableInstance(layer, torch.nn.Parameter, states)
inv_single = SingleInvariantConstant(vi)
pprint(inv_single.get_invariant_properties(), indent=2)

{ 'same': { 'name': 'module.3.mlp.dense_h_to_4h.bias',
            'properties__backward_hooks': None,
            'properties__base': None,
            'properties__cdata': 1161366720,
            'properties__grad_fn': None,
            'properties__has_symbolic_sizes_strides': False,
            'properties__hp_mapping': None,
            'properties__post_accumulate_grad_hooks': None,
            'properties__python_dispatch': False,
            'properties__version': 1,
            'properties_ds_pipe_replicated': False,
            'properties_grad_fn': None,
            'properties_is_cpu': False,
            'properties_is_cuda': True,
            'properties_is_ipu': False,
            'properties_is_leaf': True,
            'properties_is_meta': False,
            'properties_is_mkldnn': False,
            'properties_is_mps': False,
            'properties_is_mtia': False,
            'properties_is_nested': False,
            'properties_is_ort': False,
            'propert

In [8]:
# layer = 'module.3.mlp.dense_h_to_4h.bias'
variable_instances = []
for layer in traces_by_name.keys():
    state_changes = traces_by_name[layer]
    initial_state = None
    for param in trace_state_dump.iloc[0].state:
        if param['name'] == layer:
            initial_state = param
            break
    assert initial_state is not None
    if len(state_changes) != 23:
        print("Warning: layer {} has {} state changes".format(layer, len(state_changes)))
        continue
    states = construct_states(initial_state, state_changes)
    vi = VariableInstance(layer, torch.nn.Parameter, states)
    variable_instances.append(vi)
inv_multi = MultiInvariantConsistency(variable_instances)
pprint(inv_multi.get_invariant_properties(), indent=2)

{ 'consistent': [ 'properties_is_cpu',
                  'properties_is_ipu',
                  'properties_is_sparse_csr',
                  'properties__base',
                  'properties_is_xla',
                  'properties_is_quantized',
                  'properties_partition_stride',
                  'properties_is_xpu',
                  'properties_itemsize',
                  'properties_ds_pipe_replicated',
                  'properties_is_sparse',
                  'properties_is_ort',
                  'properties__backward_hooks',
                  'properties_is_mtia',
                  'properties__has_symbolic_sizes_strides',
                  'properties_is_nested',
                  'properties_grad_fn',
                  'properties_name',
                  'properties__python_dispatch',
                  'properties_retains_grad',
                  'properties_is_vulkan',
                  'properties__grad_fn',
                  'properties_is_meta',
         

In [2]:
import os, sys, tqdm, json, pandas as pd

# trace loading process
path_to_traces = "/home/yuxuan/Megatron-DeepSpeed/experiments/toy-ml-daikon-auto-syncing"

trace_paths = [os.path.join(path_to_traces, f) for f in os.listdir(path_to_traces) if f.endswith("_trace.log")]

keyword = "layernorm.bias"

def get_process_id_from_trace_path(trace_path):
    return os.path.basename(trace_path).split("_")[0]

trace_by_process_id = {}
for trace_path in trace_paths:
    process_id = get_process_id_from_trace_path(trace_path)
    with open(trace_path, "r") as f:
        lines = f.readlines()
    traces = []
    for trace in tqdm.tqdm(lines):
        if not trace.startswith("{"):
            # heuristics to skip non-json lines
            continue
        try:
            traces.append(json.loads(trace))
        except:
            print(trace)
            raise
    traces_df = pd.DataFrame(traces)
    traces_state_change = traces_df[(traces_df["type"] == "state_change")]
    trace_state_dump = traces_df[(traces_df["type"] == "state_dump")]
    # split the traces by name
    traces_by_name = {}
    for name, group in traces_state_change.groupby("name"):
        traces_by_name[name] = group.reset_index(drop=True)
    trace_by_process_id[process_id] = {
        "original_state_dump": trace_state_dump,
        "traces_by_name": traces_by_name
    }

100%|██████████| 2160262/2160262 [00:26<00:00, 80552.52it/s] 


In [23]:
"""Let's select the relevant layers with `keyword` and construct the variable instances
We are going to apply multi-invariant consistency checking on each of these layer across all processes
"""

def construct_states(initial_state, changes):
    # change "param" to "value" for consistency
    if 'param' in initial_state:
        initial_state['value'] = initial_state['param']
        del initial_state['param']
    
    states = [initial_state]
    for i, trace in changes.iterrows():
        state = states[-1].copy()
        if 'properties' in trace.change:
            state['properties'] = trace.change['properties']['new']
        if 'value' in trace.change:
            state['value'] = trace.change['value']['new']
        states.append(state)
    return states

variable_instances = {}
for process_id, trace in trace_by_process_id.items():
    traces_by_name = trace["traces_by_name"]
    original_state_dump = trace["original_state_dump"]
    for name in traces_by_name.keys():
        if keyword in name:
            state_changes = traces_by_name[name]
            initial_state = None
            for param in original_state_dump.iloc[0].state:
                if param['name'] == name:
                    initial_state = param
                    break
            assert initial_state is not None
            if len(state_changes) != 23:
                print("Warning: layer {} has {} state changes".format(name, len(state_changes)))
                continue
            states = construct_states(initial_state, state_changes)
            vi = VariableInstance(name, torch.nn.Parameter, states, len(states) * [traces_by_name['module.3.input_layernorm.bias'].meta_vars.iloc[0]])
            if name not in variable_instances:
                variable_instances[name] = []
            variable_instances[name].append(vi)

for name, vis in variable_instances.items():
    print(name, len(vis))
    inv_multi = MultiInvariantConsistency(vis[5:-1])
    # pprint(inv_multi.get_invariant_properties(), indent=2)
    pprint(inv_multi.find_preconditions(), indent=2)
    pre_conds = inv_multi.find_preconditions()
    for pre_cond in pre_conds['consistent']:
        print(pre_cond, traces_by_name[name].meta_vars.iloc[0][pre_cond])
    print(traces_by_name[name].meta_vars.iloc[0].keys())

    assert 'value' in inv_multi.get_invariant_properties()['consistent']
    break


module.3.input_layernorm.bias 8
{ 'consistent': [ '_PIPELINE_MODEL_PARALLEL_GROUP_YUXUAN_RANK',
                  '_VIRTUAL_PIPELINE_MODEL_PARALLEL_RANK',
                  '_MPU_TENSOR_MODEL_PARALLEL_RANK',
                  '_EMBEDDING_GROUP_YUXUAN_RANK',
                  '_MPU_PIPELINE_MODEL_PARALLEL_RANK',
                  '_DATA_PARALLEL_GROUP_YUXUAN_RANK']}
_PIPELINE_MODEL_PARALLEL_GROUP_YUXUAN_RANK 0
_VIRTUAL_PIPELINE_MODEL_PARALLEL_RANK None
_MPU_TENSOR_MODEL_PARALLEL_RANK None
_EMBEDDING_GROUP_YUXUAN_RANK 0
_MPU_PIPELINE_MODEL_PARALLEL_RANK None
_DATA_PARALLEL_GROUP_YUXUAN_RANK 1
dict_keys(['thread_id', 'process_id', '_TENSOR_MODEL_PARALLEL_GROUP_YUXUAN_RANK', '_PIPELINE_MODEL_PARALLEL_GROUP_YUXUAN_RANK', '_MODEL_PARALLEL_GROUP_YUXUAN_RANK', '_EMBEDDING_GROUP_YUXUAN_RANK', '_DATA_PARALLEL_GROUP_YUXUAN_RANK', '_VIRTUAL_PIPELINE_MODEL_PARALLEL_RANK', '_MPU_TENSOR_MODEL_PARALLEL_RANK', '_MPU_PIPELINE_MODEL_PARALLEL_RANK', '_PIPELINE_GLOBAL_RANKS'])


In [12]:
type(traces_by_name['module.3.input_layernorm.bias'].meta_vars.iloc[0])

dict

In [13]:
2 * [1]

[1, 1]